## 작업형3

### 문제1. 주어진 조개 데이터 300개 중 앞에서부터 210개는 train 데이터로 만들고, 나머지 90개는 test데이터로 만든다. 모델을 학습(적합)할 때는 train데이터를 사용하고, 예측할 때는 test데이터를 사용한다. 모델은 로지스틱 회귀를 써서 성별(gender)을 예측하되, 패널티는 부과하지 않는다.
- 종속변수: gender(0:암컷, 1:수컷)

### 문제1-1. weight를 독립변수로 gender를 종속변수로 사용하여 로지스틱 회귀 모형을 만들고, weight 변수가 한 단위 증가할 때 수컷일 오즈비 값은? (반올림하여 소수 넷째자리까지 계산)

In [25]:
import pandas as pd
df = pd.read_csv("clam.csv")

#print(df)

df210 = df.iloc[:210]
test = df[210:]
#print(df210)
print(df210.shape, test.shape)

#로지스틱회귀 - 분류
from statsmodels.formula.api import logit #로지스틱회귀분석 import
# print(dir(logit))
# print(help(logit))
# 순서는 종속변수 우선, 그 후 독립변수를 작성 (~로 구분)
model = logit('gender ~ weight', data = df210).fit()
# 학습시에는 df210을 사용하여 학습
print(model.summary())
# 학습용으로는 train, 예측에는 test 사용.

# weight의 coef(회귀계수)는 0.0047
import numpy as np
np.exp(0.0047) #1.0047110623241844

model.params['weight'] #0.004706195937284832
print(round(np.exp(model.params['weight']),4)) #np.exp(0.0047과 근사)



(210, 6) (90, 6)
Optimization terminated successfully.
         Current function value: 0.690045
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:                 gender   No. Observations:                  210
Model:                          Logit   Df Residuals:                      208
Method:                           MLE   Df Model:                            1
Date:                Sun, 16 Jun 2024   Pseudo R-squ.:                0.003431
Time:                        08:54:50   Log-Likelihood:                -144.91
converged:                       True   LL-Null:                       -145.41
Covariance Type:            nonrobust   LLR p-value:                    0.3178
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.3140      0.276     -1.137      0.256      -0.855       0.227
weight      

### 문제1-2. gender를 종속변수로 하고 나머지 변수들(age, length, diameter, height, weight)을 독립변수로 사용하는 로지스틱 회귀 모델을 적합시킨 후, 잔차 이탈도(Residual Deviance)를 계산하시오. (반올림하여 소수 둘째자리까지 계산)

In [36]:
# model = logit('gender ~ age + length + diameter + height + weight', data = df210).fit()
#print(model.summary())

import statsmodels.api as sm
from statsmodels.formula.api import glm
formula = "gender ~ age + length + diameter + height + weight"
model = glm(formula, data = df210, family = sm.families.Binomial()).fit() # 이진분류
#print(dir(glm))
#print(model.summary())
# Deviance: 286.93 (잔차이탈도)
#print(model.deviance) # 286.93267518507366
print(round(model.deviance,2))


286.93


### 문제1-3. 독립변수 weight 만 사용해 학습한 로지스틱 회귀모델에서 test데이터의 gender를 예측하고, error rate(오류율)를 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [46]:
from sklearn.metrics import accuracy_score
from statsmodels.formula.api import logit

model = logit('gender ~ weight', data = df210).fit()
# target = test.pop('gender')
pred = model.predict(test) >0.5
print(model.predict(test) > 0.5) # 일반적으로 오류율은 0.5를 기준으로 하므로,
# 0.5 조건을 붙여 True와 False로 구분.
# print(model.summary())
acc = accuracy_score(target, pred)
errorrate = 1 - acc
print(round(errorrate,3))


Optimization terminated successfully.
         Current function value: 0.690045
         Iterations 4
210     True
211     True
212     True
213     True
214    False
       ...  
295    False
296    False
297    False
298    False
299    False
Length: 90, dtype: bool
0.478


### 문제2-1. ERP와 가장 상관관계 높은값을 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [47]:
import pandas as pd
df = pd.read_csv("system.csv")

In [58]:
# print(abs(df.corr())) #0.434442
print(round(0.434442,3))



               ERP  Feature1  Feature2  Feature3       CPU
ERP       1.000000  0.434442  0.076116  0.059888  0.201027
Feature1  0.434442  1.000000  0.076148  0.049980  0.098571
Feature2  0.076116  0.076148  1.000000  0.126589  0.071456
Feature3  0.059888  0.049980  0.126589  1.000000  0.035247
CPU       0.201027  0.098571  0.071456  0.035247  1.000000
0.434


### 문제2-2. CPU 컬럼이 100미만인 것만 찾아 ERP를 종속 변수로, 나머지 변수들을 독립 변수로 설정해 선형회귀 모델을 만들고 적합한 결정계수를 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [66]:
#df
cond1 = df['CPU'] < 100
df[cond1] #cpu 컬럼이 100 미만


from statsmodels.formula.api import ols
model = ols('ERP ~ Feature1 + Feature2 + Feature3 + CPU', data = df[cond1]).fit()
print(model.summary())
#결정계수는 r-square로 구함.0.226
# 수정된 결정계수는 Adj. R-squared로 구함.

# print(model.rsquared) 더 정확한 값을 알고 싶을 때. 기본적으로는 필요하지 않음.



                            OLS Regression Results                            
Dep. Variable:                    ERP   R-squared:                       0.226
Model:                            OLS   Adj. R-squared:                  0.193
Method:                 Least Squares   F-statistic:                     6.937
Date:                Sun, 16 Jun 2024   Prob (F-statistic):           6.07e-05
Time:                        09:15:45   Log-Likelihood:                -429.44
No. Observations:                 100   AIC:                             868.9
Df Residuals:                      95   BIC:                             881.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     74.4348     15.094      4.932      0.0

### 문제 2-3. 2-2에서 만든 모델에서 독립변수 중 p-value가 가장 높은값을 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [70]:
# p-value 역시 model.summary()로 확인 가능.

# print(model.summary())

#단, 소숫점 셋째 자리까지만 표시되니 정확히 확인을 위해 따로 출력
# print(model.pvalues) #0.456708
print(round(max(model.pvalues),3)) #0.457

# coef 회귀계수 P>|t| p-value 0.025 ~ 0.975 신뢰구간

                            OLS Regression Results                            
Dep. Variable:                    ERP   R-squared:                       0.226
Model:                            OLS   Adj. R-squared:                  0.193
Method:                 Least Squares   F-statistic:                     6.937
Date:                Sun, 16 Jun 2024   Prob (F-statistic):           6.07e-05
Time:                        09:17:33   Log-Likelihood:                -429.44
No. Observations:                 100   AIC:                             868.9
Df Residuals:                      95   BIC:                             881.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     74.4348     15.094      4.932      0.0